In [1]:
import os
import os.path as op
from glob import glob
import numpy as np
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

_y_max_parking = 0.66
frames_to_check = 200
threshold = 0.05
frames_per_second = 20

In [2]:
def extract_subclip(path):
    car = {}
    max_frame = 0


    with open(path) as f:
        for line in f.readlines():
            line = line.split()
            frame = int(line[0])
            if frame > max_frame:
                max_frame = frame
            idx = line[1]
            left = int(line[2])
            top = int(line[3])
            w = int(line[4])
            h = int(line[5])
            W = int(line[6])
            H = int(line[7])
            W, H = 2560, 1440 ###########
            _x = (left+w/2)/W
            _y = (top+h/2)/H
            _w = w/W
            _h = h/H
            try:
                car[idx]['position'][frame] = [_x,_y,_w,_h]
                if _y+_h/2 > _y_max_parking:
                    car[idx]['parking'] = True
                if (frame-frames_to_check) in car[idx]['position']:
                    _x_acc = _x - car[idx]['position'][frame-frames_to_check][0]
                    _y_acc = _y - car[idx]['position'][frame-frames_to_check][1]
                    car[idx]['acceleration'][int(frame-frames_to_check/2)] = np.sqrt(_x_acc**2+_y_acc**2)
            except Exception as e:
                print(e)
                car[idx] = {}
                car[idx]['position'] = {}
                car[idx]['position'][frame] = [_x,_y,_w,_h]
                car[idx]['parking'] = False
                car[idx]['acceleration'] = {}
                
                
                
    for idx in car:
        if car[idx]['parking'] == False:
            continue
        check_frames = []
        for frame in car[idx]['acceleration']:
            before_acc = 0
            after_acc = 0
            if int(frame-frames_to_check/2) in car[idx]['acceleration']:
                before_acc = car[idx]['acceleration'][int(frame-frames_to_check/2)]
            if int(frame+frames_to_check/2) in car[idx]['acceleration']:
                after_acc = car[idx]['acceleration'][int(frame+frames_to_check/2)]
            if (before_acc>threshold) and (after_acc>threshold):
                check_frames.append(frame)
        if len(check_frames) == 0:
            continue
        start = max(min(check_frames)-frames_to_check,0)
        stop = min(max(check_frames)+frames_to_check,max_frame)
        i = 1
        path_video = path.replace('results','extracted_videos').replace('txt','mp4')
        folder_save_video = op.dirname(path.replace('results','results2'))
        os.makedirs(folder_save_video,exist_ok=True)
        while True:
            path_save_video = path.replace('results','results2').replace('.txt',f'_{i}.mp4')
            if not op.exists(path_save_video):
                print(f'Cut video {path_video}')
                print(f'Save video to {path_save_video}')
                ffmpeg_extract_subclip(path_video, 
                                       start/frames_per_second, 
                                       stop/frames_per_second, 
                                       targetname=path_save_video)
                break
            else:
                i += 1

In [ ]:
sources = '/data/results/*'
paths = glob(sources)
while len(paths) != 0:
    for path in paths:
        if path[-4:] != '.txt':
            continue
        extract_subclip(path)
    sources = op.join(sources,'*')
    paths = glob(sources)